In [ ]:
## data
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
from unicodedata import normalize

## vis
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# contagem de aging
from collections import Counter

from IPython.display import display, HTML

## modelo
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# check xgboost version
from xgboost import XGBClassifier

## alertas
import warnings
warnings.filterwarnings("ignore")

***
### Funções

In [ ]:
def just_check_nan(df):
    # Check NaN Pandas and Numpy
    missing_val_count_by_column = (df.isna().sum())

    columns_with_nan = missing_val_count_by_column[missing_val_count_by_column > 0]
    print('Dados com NaN:')
    print(columns_with_nan)
    print('')
    print('columns_with_nan.shape', columns_with_nan.shape)
    print('')

In [ ]:
def plot_feature_freq(df, feature):
    labels, counts = np.unique(df[feature].dropna(), return_counts=True)
    total = counts[0] + counts[1]
    plt.figure(figsize=(10, 7))
    ax = sns.barplot(x=labels, y=(counts/total)*100, data = df)
    ax2 = sns.barplot(x=labels, y=counts, data = df)

    for p in ax.patches:
        ax.annotate('{:.2f}%'.format(p.get_height()), (p.get_x() + 0.3, p.get_height()+0.6), rotation = 0)

#     ax.set(xlabel='Trocou', ylabel='Porcentagem')
    ax.xaxis.label.set_size(20)
    ax.yaxis.label.set_size(20)
    plt.show()

In [ ]:
def plot_feature_freq_mes(df, feature):
    labels, counts = np.unique(df[feature].dropna(), return_counts=True)
    total = counts[0] + counts[1] + counts[2] + counts[3] + counts[4] + counts[5] + counts[6]
    plt.figure(figsize=(10, 7))
    ax = sns.barplot(x=labels, y=(counts/total)*100, data = df)
    ax2 = sns.barplot(x=labels, y=counts, data = df)

    for p in ax.patches:
        ax.annotate('{:.2f}%'.format(p.get_height()), (p.get_x() + 0.3, p.get_height()+0.6), rotation = 0)

#     ax.set(xlabel='Trocou', ylabel='Porcentagem')
    ax.xaxis.label.set_size(20)
    ax.yaxis.label.set_size(20)
    plt.show()

*****
## Coleta de dados

In [ ]:
FILE = "/kaggle/input/bigdata/batdtfim_full.csv"

In [ ]:
client_chunks = []
dados_teste = pd.read_csv(FILE, sep = ";", 
                          header = 0, 
                          index_col = False, 
                          names = None, 
                          low_memory = False, 
                          chunksize = 10**5) 

In [ ]:
for chunk in dados_teste:

    new_clients = pd.DataFrame(chunk)

    # filter data chunk with frac percentage
#     new_clients = new_clients.sample(frac = 0.01)
    new_clients = new_clients.drop_duplicates()

    # saves to list
    client_chunks.append(new_clients)

dados = pd.concat(client_chunks).reset_index(drop = True)

In [ ]:
# FILE = "/kaggle/input/datasetsnew/batdtfim500kage.csv"
# dados = pd.read_csv(FILE, sep = ';', encoding = 'ISO-8859-1', engine ='python')

dados = dados.rename(columns={'a.msisdn': 'msisdn'})\
             .rename(columns={'a.imsi': 'imsi'})\
             .rename(columns={'a.tac': 'tac'})\
             .rename(columns={'a.dt_inicio': 'dt_inicio'})\
             .rename(columns={'a.dt_fim': 'dt_fim'})\
             .rename(columns={'a.aging_aparelho': 'aging_aparelho'})\
             .rename(columns={'a.cat_aparelho': 'cat_aparelho'})\
             .rename(columns={'a.trocou_aparelho': 'trocou_aparelho'})\
             .rename(columns={'a.fabricante': 'fabricante'})\
             .rename(columns={'a.modelo': 'modelo'})\
             .rename(columns={'a.imei': 'imei'})\
             .rename(columns={'a.ranking': 'ranking'})\
             .rename(columns={'a.cpfidentifier': 'cpfidentifier'})\
             .rename(columns={'a.subscriberkey': 'subscriberkey'})\
             .rename(columns={'a.customerkey': 'customerkey'})\
             .rename(columns={'a.customersubtypedesc': 'customersubtypedesc'})\
             .rename(columns={'a.subscriberstatuskey': 'subscriberstatuskey'})\
             .rename(columns={'a.offertechnology': 'offertechnology'})\
             .rename(columns={'a.citydesc': 'citydesc'})\
             .rename(columns={'a.statedesc': 'statedesc'})\
             .rename(columns={'a.subscribermainofferdesc': 'subscribermainofferdesc'})\
             .rename(columns={'a.assignedchargeamount': 'assignedchargeamount'})\
             .rename(columns={'a.fl_3g': 'fl_3g'})\
             .rename(columns={'a.fl_4g': 'fl_4g'})\
             .rename(columns={'a.fl_5g': 'fl_5g'})\
             .rename(columns={'a.dt_ano_stts': 'dt_ano_stts'})\
             .rename(columns={'a.idade': 'idade'})\
             .rename(columns={'a.avg_msisdn': 'avg_msisdn'})\
             .rename(columns={'a.avg_cpf': 'avg_cpf'})\
#              .drop_duplicates().reset_index(drop=True)

dados

### Limpeza dos dados

In [ ]:
just_check_nan(dados)

In [ ]:
dados_droped = dados.dropna(subset=['imsi',
                                    'statedesc', 
                                    'subscribermainofferdesc', 
                                    'assignedchargeamount', 'fl_3g', 'fl_4g', 'fl_5g', 'dt_ano_stts'])\
                    .reset_index(drop=True)
# dados_droped

In [ ]:
just_check_nan(dados_droped)

In [ ]:
# plot_feature_freq(df, feature, qt_cat)
plot_feature_freq(dados_droped, 'trocou_aparelho')

### Equilibrando classes

In [ ]:
dados['dt_ano_stts'] = np.asarray(dados['dt_ano_stts'], dtype = 'int16')
dados['idade'] = np.asarray(dados['idade'], dtype = 'int16')
dados['fl_3g'] = np.asarray(dados['fl_3g'], dtype = 'int8')
dados['fl_4g'] = np.asarray(dados['fl_4g'], dtype = 'int8')
dados['fl_5g'] = np.asarray(dados['fl_5g'], dtype = 'int8')

In [ ]:
dados.info()

In [ ]:
dados.columns

***
### Analise individual

In [ ]:
dados_droped['trocou_aparelho'].unique()

In [ ]:
dados_droped['cat_aparelho'].unique()

In [ ]:
dados_droped['aging_aparelho'].unique()

In [ ]:
dados_droped['customersubtypedesc'].unique()

In [ ]:
dados_droped['subscriberstatuskey'].unique()

In [ ]:
dados_droped['ranking'].unique()

In [ ]:
dados_droped['fl_3g'].equals(dados_droped['fl_4g'])

In [ ]:
dados_droped['fl_3g'].equals(dados_droped['fl_5g'])

In [ ]:
dados_droped['fl_4g'].equals(dados_droped['fl_5g'])

In [ ]:
len(dados_droped['fabricante'].describe())

In [ ]:
# Tamanho do gráfico em polegadas
plt.figure(figsize =(9, 3))

#Plotando o boxplot das espécies em relação ao tamanho das sépalas
bplots = plt.boxplot(dados_droped['idade'], vert = 0)

# Adicionando Título ao gráfico
plt.title("Boxplot da Coluna idade", loc="center", fontsize=18)
plt.xlabel("Anos")
plt.ylabel("idade")

plt.show()

In [ ]:
dados_droped['dt_ano_stts'].describe()

In [ ]:
# Tamanho do gráfico em polegadas
plt.figure(figsize =(9, 3))

#Plotando o boxplot das espécies em relação ao tamanho das sépalas
bplots = plt.boxplot(dados_droped['dt_ano_stts'], vert = 0)

# Adicionando Título ao gráfico
plt.title("Boxplot da Coluna dt_ano_stts", loc="center", fontsize=18)
plt.xlabel("Anos")
plt.ylabel("dt_ano_stts")

plt.show()

In [ ]:
dados_droped['avg_msisdn'].describe()

In [ ]:
# Tamanho do gráfico em polegadas
plt.figure(figsize =(9, 3))

#Plotando o boxplot das espécies em relação ao tamanho das sépalas
bplots = plt.boxplot(dados_droped['avg_msisdn'], vert = 0)

# Adicionando Título ao gráfico
plt.title("Boxplot da Coluna avg_msisdn", loc="center", fontsize=18)
plt.xlabel("Média")
plt.ylabel("avg_msisdn")

plt.show()

In [ ]:
dados_droped['avg_cpf'].max()

In [ ]:
# Tamanho do gráfico em polegadas
plt.figure(figsize =(9, 3))

#Plotando o boxplot das espécies em relação ao tamanho das sépalas
bplots = plt.boxplot(dados_droped['avg_cpf'], vert = 0)

# Adicionando Título ao gráfico
plt.title("Boxplot da Coluna avg_cpf", loc="center", fontsize=18)
plt.xlabel("Média")
plt.ylabel("avg_cpf")

plt.show()

In [ ]:
# dados_droped['peso_aparelho'].unique()

In [ ]:
# dados_droped['update_software'].unique()

In [ ]:
dados_droped['assignedchargeamount'].describe()

In [ ]:
# Tamanho do gráfico em polegadas
plt.figure(figsize =(11, 6))

#Plotando o boxplot das espécies em relação ao tamanho das sépalas
bplots = plt.boxplot(dados_droped['assignedchargeamount'],  vert = 0, patch_artist = False)

# Adicionando Título ao gráfico
plt.title("Boxplot dos valores de faturas", loc="center", fontsize=18)
plt.xlabel("Valores")
plt.ylabel("assignedchargeamount")

plt.show()

***
## Data Analysis

In [ ]:
# from pandas_profiling import ProfileReport

In [ ]:
# dados_analysis = ProfileReport(dados_droped, title="Profiling Report", progress_bar=True, infer_dtypes=False)
# profile_td_html = dados_analysis.to_html()

In [ ]:
# display(HTML(profile_td_html))